In [1]:
import warnings
warnings.simplefilter("ignore")

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [2]:
# Import data

DATA_PATH = 'Reservoir_Project/Data'

In [3]:
basin_inflow = pd.read_excel(f'{DATA_PATH}/Custom/basin_inflow_no_dates.xlsx', index_col=0)

In [4]:
basin_inflow.head()

,INFLOW,ADR_PRECIP_ACC,ADR_PRECIP_INCR,ADR_TEMP_AVG,ADR_TEMP_MAX,ADR_TEMP_MIN,HYS_PRECIP_ACC,HYS_PRECIP_INCR,HYS_SNOW_DEPTH,HYS_SNOW_WATER_CONTENT,...,FRN_SNOW_DEPTH,FRN_SNOW_WATER_CONTENT,FRN_TEMP_AVG,FRN_TEMP_MAX,FRN_TEMP_MIN,PFH_PRECIP_ACC,PFH_PRECIP_INCR,PFH_TEMP_AVG,PFH_TEMP_MAX,PFH_TEMP_MIN
0,27.666,9.52,0.00,44,58,34,20.24,0.00,38,7.67,...,30,7.90,44,55,37,7.72,0.04,40,61,30
1,30.044,9.52,0.00,47,57,39,20.36,0.12,38,7.79,...,30,7.88,38,46,32,7.76,0.04,39,63,29
2,48.478,9.76,0.24,45,50,38,20.36,0.00,39,7.79,...,30,7.86,30,35,25,7.76,0.00,40,53,29
3,90.246,11.28,1.52,46,50,42,25.76,5.40,45,8.15,...,33,8.27,31,33,28,11.00,3.24,39,47,32
4,200.200,11.96,0.68,41,46,37,27.44,1.68,58,11.75,...,33,8.27,31,33,28,12.04,1.04,33,38,32


In [5]:
basin_inflow.columns

Index(['INFLOW', 'ADR_PRECIP_ACC', 'ADR_PRECIP_INCR', 'ADR_TEMP_AVG',
       'ADR_TEMP_MAX', 'ADR_TEMP_MIN', 'HYS_PRECIP_ACC', 'HYS_PRECIP_INCR',
       'HYS_SNOW_DEPTH', 'HYS_SNOW_WATER_CONTENT', 'HYS_TEMP_AVG',
       'HYS_TEMP_MAX', 'HYS_TEMP_MIN', 'DUN_PRECIP_ACC', 'DUN_WIND_GUST',
       'DUN_WIND_SPEED', 'DUN_TEMP_AVG', 'DUN_TEMP_MAX', 'DUN_TEMP_MIN',
       'SGP_PRECIP_ACC', 'SGP_PRECIP_INCR', 'SGP_TEMP_AVG', 'SGP_TEMP_MAX',
       'SGP_TEMP_MIN', 'FRN_PRECIP_ACC', 'FRN_PRECIP_INCR', 'FRN_SNOW_DEPTH',
       'FRN_SNOW_WATER_CONTENT', 'FRN_TEMP_AVG', 'FRN_TEMP_MAX',
       'FRN_TEMP_MIN', 'PFH_PRECIP_ACC', 'PFH_PRECIP_INCR', 'PFH_TEMP_AVG',
       'PFH_TEMP_MAX', 'PFH_TEMP_MIN'],
      dtype='object')

#### Exponential moving average smoothing

In [6]:
"""
Exponential moving average smoothing

- Uses past 30 periods or days to inform prediction (give them more weight) in order to capture trends
- Span corresponds to what is commonly called an “N-day EW moving average”
"""

def smooth(df):
  for col in df.columns:
    df[col] = df[col].ewm(span=30.0, ignore_na=True).mean(engine='numba')
  
  return df 

In [7]:
smoothed_basin_inflow = smooth(basin_inflow.copy())
smoothed_basin_inflow.head()

,INFLOW,ADR_PRECIP_ACC,ADR_PRECIP_INCR,ADR_TEMP_AVG,ADR_TEMP_MAX,ADR_TEMP_MIN,HYS_PRECIP_ACC,HYS_PRECIP_INCR,HYS_SNOW_DEPTH,HYS_SNOW_WATER_CONTENT,...,FRN_SNOW_DEPTH,FRN_SNOW_WATER_CONTENT,FRN_TEMP_AVG,FRN_TEMP_MAX,FRN_TEMP_MIN,PFH_PRECIP_ACC,PFH_PRECIP_INCR,PFH_TEMP_AVG,PFH_TEMP_MAX,PFH_TEMP_MIN
0,27.666000,9.520000,0.000000,44.000000,58.000000,34.000000,20.240000,0.000000,38.000000,7.670000,...,30.000000,7.900000,44.000000,55.000000,37.000000,7.720000,0.040000,40.000000,61.000000,30.000000
1,28.894633,9.520000,0.000000,45.550000,57.483333,36.583333,20.302000,0.062000,38.000000,7.732000,...,30.000000,7.889667,40.900000,50.350000,34.416667,7.740667,0.040000,39.483333,62.033333,29.483333
2,35.862281,9.605391,0.085391,45.354313,54.820807,37.087375,20.322636,0.039941,38.355794,7.752636,...,30.000000,7.879111,37.021844,44.888560,31.066272,7.747545,0.025768,39.667160,58.819326,29.311366
3,50.846977,10.066807,0.480678,45.532223,53.492499,38.440982,21.820828,1.516832,40.186515,7.862124,...,30.826609,7.986815,35.362606,41.612828,30.221402,8.643715,0.911406,39.483333,55.562671,30.052183
4,84.828406,10.497554,0.526028,44.501033,51.787774,38.113124,23.099326,1.553957,44.239514,8.746710,...,31.321108,8.051247,34.370008,39.653201,29.715979,9.416452,0.940664,38.008218,51.566738,30.495358


#### Remove seasonality

In [8]:
"""
Make data stationary

Seasonal data is differenced by substracting an observation from the same time in the previous cycle.
"""

def difference(dataset, interval=1):
    cols = []
    for col in dataset.columns: 
        col_data = dataset[col]
        series = col_data.values
        diff = list()
        for i in range(interval, len(series)):
            value = series[i] - series[i - interval]
            diff.append(value)
        cols.append(pd.Series(diff))
    
    return pd.concat(cols, axis=1)

In [9]:
days = 365

stationary_basin_inflow = difference(smoothed_basin_inflow, days)
stationary_basin_inflow.index = smoothed_basin_inflow.index[days:]
stationary_basin_inflow.columns = smoothed_basin_inflow.columns
stationary_basin_inflow.reset_index(drop=True, inplace=True)
print("Null/NaN count: ", stationary_basin_inflow.isnull().sum().sum())

Null/NaN count:  0


In [10]:
stationary_basin_inflow.head()

,INFLOW,ADR_PRECIP_ACC,ADR_PRECIP_INCR,ADR_TEMP_AVG,ADR_TEMP_MAX,ADR_TEMP_MIN,HYS_PRECIP_ACC,HYS_PRECIP_INCR,HYS_SNOW_DEPTH,HYS_SNOW_WATER_CONTENT,...,FRN_SNOW_DEPTH,FRN_SNOW_WATER_CONTENT,FRN_TEMP_AVG,FRN_TEMP_MAX,FRN_TEMP_MIN,PFH_PRECIP_ACC,PFH_PRECIP_INCR,PFH_TEMP_AVG,PFH_TEMP_MAX,PFH_TEMP_MIN
0,8.944327,-4.372491,0.151906,0.121939,-6.413691,4.031741,-7.699793,0.499151,-24.334203,-5.572962,...,-17.288084,-1.700984,-11.105608,-14.746997,-9.483165,-0.778717,0.202783,-1.782846,-9.608046,0.235409
1,9.341866,-4.198782,0.178235,-1.693993,-6.386464,1.123780,-7.166968,0.546883,-21.861028,-5.259932,...,-16.559821,-1.572523,-8.127827,-10.629448,-6.804144,-0.470434,0.295506,-1.667286,-11.569893,0.672372
2,3.619606,-4.121670,0.081345,-1.940629,-4.052768,0.122505,-6.579541,0.581273,-19.903208,-4.836831,...,-15.878542,-1.451461,-4.364004,-5.666108,-3.364234,-0.164425,0.293254,-2.484406,-9.030946,0.189132
3,-11.380245,-4.431068,-0.324699,-2.338777,-2.903043,-1.373675,-7.501159,-0.927955,-19.827644,-4.531210,...,-16.067826,-1.455787,-2.811724,-2.856341,-2.435625,-0.765312,-0.610386,-2.634950,-6.018703,-0.842039
4,-45.496431,-4.719605,-0.380113,-1.320067,-1.107315,-1.114675,-8.229958,-0.992749,-22.226377,-5.027467,...,-14.417730,-1.423511,-1.661118,-0.622939,-1.787349,-1.259236,-0.656484,-1.150053,-1.864316,-1.169740


#### Normalization

In [24]:
"""
Normalizing data

Maximum absolute scaling rescales each feature between -1 and 1 by dividing every observation 
by its maximum absolute value.
"""

def normalize(df):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    normalized_df = scaler.fit_transform(df)

    return normalized_df

'\ndef normalize(df):\n    for col in df.columns:\n        df[col] = df[col] / df[col].abs().max()\n\n    return df\n'

In [25]:
normalized_df = normalize(stationary_basin_inflow)

In [26]:
normalized_basin_inflow = pd.DataFrame(normalized_df, columns=basin_inflow.columns) 

In [27]:
normalized_basin_inflow.head()

,INFLOW,ADR_PRECIP_ACC,ADR_PRECIP_INCR,ADR_TEMP_AVG,ADR_TEMP_MAX,ADR_TEMP_MIN,HYS_PRECIP_ACC,HYS_PRECIP_INCR,HYS_SNOW_DEPTH,HYS_SNOW_WATER_CONTENT,...,FRN_SNOW_DEPTH,FRN_SNOW_WATER_CONTENT,FRN_TEMP_AVG,FRN_TEMP_MAX,FRN_TEMP_MIN,PFH_PRECIP_ACC,PFH_PRECIP_INCR,PFH_TEMP_AVG,PFH_TEMP_MAX,PFH_TEMP_MIN
0,0.044963,-0.289112,0.340301,0.054044,-0.375165,0.305438,-0.011090,0.388324,-0.123764,0.061586,...,0.108819,0.183304,-0.491096,-0.610589,-0.458829,-0.078244,0.224818,-0.039676,-0.564907,0.045612
1,0.045396,-0.283597,0.376414,-0.093752,-0.373458,0.056615,-0.006376,0.419682,-0.106719,0.069362,...,0.113669,0.185770,-0.363760,-0.448188,-0.336938,-0.073231,0.299318,-0.030774,-0.682164,0.086839
2,0.039165,-0.281148,0.243520,-0.113826,-0.227147,-0.029060,-0.001178,0.442274,-0.093225,0.079873,...,0.118205,0.188094,-0.202812,-0.252427,-0.180427,-0.068255,0.297508,-0.093725,-0.530415,0.041245
3,0.022834,-0.290972,-0.313413,-0.146230,-0.155065,-0.157082,-0.009333,-0.549211,-0.092705,0.087465,...,0.116945,0.188011,-0.136433,-0.141606,-0.138177,-0.078026,-0.428531,-0.105323,-0.350378,-0.056045
4,-0.014312,-0.300133,-0.389419,-0.063319,-0.042481,-0.134920,-0.015781,-0.591777,-0.109237,0.075137,...,0.127933,0.188631,-0.087231,-0.053518,-0.108681,-0.086058,-0.465569,0.009074,-0.102076,-0.086963


#### Split data

In [28]:
# 70/20/10 split for training, validation, and test sets

df_size = len(normalized_basin_inflow)

train_df = normalized_basin_inflow[0:int(df_size * 0.7)] # 70% 

val_df = normalized_basin_inflow[int(df_size * 0.7):int(df_size * 0.9)] # next 20%
val_df.reset_index(drop=True, inplace=True)

test_df = normalized_basin_inflow[int(df_size * 0.9):] # last 10%
test_df.reset_index(drop=True, inplace=True)

train_df.to_excel(f'{DATA_PATH}/Custom/basin_inflow_train.xlsx') 
val_df.to_excel(f'{DATA_PATH}/Custom/basin_inflow_validation.xlsx') 
test_df.to_excel(f'{DATA_PATH}/Custom/basin_inflow_test.xlsx') 